In [1]:
import json
def round_floats(data):
    if isinstance(data, dict):
        for key, value in data.items():
            data[key] = round_floats(value)
    elif isinstance(data, list):
        for i in range(len(data)):
            data[i] = round_floats(data[i])
    elif isinstance(data, float):
        data = round(data, 3)
    return data

In [2]:
def json_to_txt(json_file, output_file, image_width, image_height):
    # JSON dosyasını oku
    with open(json_file, 'r') as file:
        try:
            data = json.load(file)
            data = round_floats(data)
        except json.JSONDecodeError as e:
            print("JSONDecodeError:", e)
            return

        # Frame ve hedef ID'leri içeren bir liste oluştur
        frame_data = []

        # JSON verilerini işle
        target_id_counter = 1
        for item in data:
            for annotation in item['annotations']:
                for result in annotation['result']:
                    sequence = result['value']['sequence']
                    for data_point in sequence:
                        frame_index = data_point.get('frame', '')
                        bbox_left = data_point.get('x', '') / 100 * image_width
                        bbox_top = data_point.get('y', '') / 100 * image_height
                        bbox_width = data_point.get('width', '') / 100 * image_width
                        bbox_height = data_point.get('height', '') / 100 * image_height
                        score = 1  # Or your score calculation logic here
                        object_category = 1
                        time = round(data_point.get('time', ''), 3)
                        target_id = target_id_counter

                        frame_data.append((frame_index, target_id, bbox_left, bbox_top, bbox_width, bbox_height, score, object_category, time))
                    target_id_counter += 1

        # Frame'leri sıralı hale getir
        frame_data.sort(key=lambda x: x[0])

        # TXT dosyasını oluştur
        with open(output_file, 'w') as f:
            # Başlık satırını yaz
            f.write("frame_index,target_id,bbox_left,bbox_top,bbox_width,bbox_height,score,object_category,time\n")

            # Sıralanmış verileri yaz
            for frame in frame_data:
                f.write(f"{frame[0]},{frame[1]},{frame[2]:.3f},{frame[3]:.3f},{frame[4]:.3f},{frame[5]:.3f},{frame[6]},{frame[7]},{frame[8]:.3f}\n")

In [3]:
json_to_txt('project-2-at-2024-06-06-17-05-cf5027b1.json','output.txt',1280,720)

In [5]:
import cv2
import pandas as pd

# Txt dosyasını oku ve DataFrame'e yükle
df = pd.read_csv('data/segment_2/data_2.txt')

# Video dosyası oluştur
output_video = cv2.VideoWriter('output_video.avi', cv2.VideoWriter_fourcc(*'DIVX'), 30, (1280, 720))

# Her bir çerçeveyi videoya dönüştür
for _, row in df.iterrows():
    frame_index = row['frame_index']
    bbox_left = int(row['bbox_left'])
    bbox_top = int(row['bbox_top'])
    bbox_width = int(row['bbox_width'])
    bbox_height = int(row['bbox_height'])
    category = int(row['object_category'])
    track_id = int(row['target_id'])

    # Çerçeveyi al ve gerekli düzenlemeleri yap
    frame_index_int = int(frame_index)
    frame_path = f'data/segment_2/{frame_index_int - 1:06d}.jpg'
    frame = cv2.imread(frame_path)
    cv2.rectangle(frame, (bbox_left, bbox_top), (bbox_left + bbox_width, bbox_top + bbox_height), (0, 255, 0), 2)
    cv2.putText(frame, f"Track ID: {track_id}", (bbox_left, bbox_top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    cv2.putText(frame, f"Category: {category}", (bbox_left, bbox_top - 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Çerçeveyi videoya ekleyin
    output_video.write(frame)

# Videoyu kapat
output_video.release()

FileNotFoundError: [Errno 2] No such file or directory: 'data/segment_2/data_2.txt'

In [3]:
import os
import cv2
import pandas as pd

def create_segments(txt_file, video_file, output_dir, segment_duration=10, fps=30):
    # Video capture object
    cap = cv2.VideoCapture(video_file)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    total_duration = frame_count / fps

    # Reading the txt file
    df = pd.read_csv(txt_file)

    segment_frames = segment_duration * fps
    num_segments = int(total_duration / segment_duration)

    for segment_index in range(num_segments):
        start_time = segment_index * segment_duration
        end_time = start_time + segment_duration
        segment_data = df[(df['time'] >= start_time) &
                          (df['time'] < end_time)].copy()

        # Resetting the frame index for each segment
        segment_data['frame_index'] = segment_data['frame_index'] - segment_data['frame_index'].min() + 1

        segment_output_dir = os.path.join(output_dir, f"segment_{segment_index+1}")
        img_output_dir = os.path.join(segment_output_dir, "img")
        segment_gt_dir=os.path.join(segment_output_dir,'gt')
        os.makedirs(img_output_dir, exist_ok=True)
        os.makedirs(segment_gt_dir, exist_ok=True)


        segment_txt_path = os.path.join(segment_gt_dir, "gt.txt")
        segment_data.to_csv(segment_txt_path, index=False, header=False)

        seqinfo_path = os.path.join(segment_output_dir, "seqinfo.ini")
        with open(seqinfo_path, "w") as seqinfo_file:
            seqinfo_file.write(f"[Sequence]\n")
            seqinfo_file.write(f"name=segment_{segment_index+1}\n")
            seqinfo_file.write(f"imDir=img\n")
            seqinfo_file.write(f"frameRate={fps}\n")
            seqinfo_file.write(f"seqLength={segment_frames}\n")
            seqinfo_file.write(f"imWidth=1280\n")
            seqinfo_file.write(f"imHeight=720\n")
            seqinfo_file.write(f"imExt=.jpg\n")

        cap.set(cv2.CAP_PROP_POS_FRAMES, start_time * fps)
        for frame_index in range(1, segment_frames + 1):
            ret, frame = cap.read()
            if not ret:
                break

            output_frame_name = os.path.join(img_output_dir, f"{frame_index:06d}.jpg")
            cv2.imwrite(output_frame_name, frame)

    cap.release()
    print("Processing complete.")

In [23]:
create_segments('output.txt', 'Coconut.mp4', 'data')

Processing complete.


In [4]:
import os
import os.path as osp
import json
from collections import defaultdict

def generate_dataset_structure(base_path,fps=30,width=1280,height=720, split_data=False):
    # Sabit kategori bilgisi
    category_info = {"id": 1, "name": "coconat"}

    # Dataset yapıları
    train_dataset = {
        "categories": [category_info],
        "annotations": [],
        "images": [],
        "videos": [],
        "num_instances": 0
    }

    test_dataset = {
        "categories": [category_info],
        "annotations": [],
        "images": [],
        "videos": [],
        "num_instances": 0
    }

    video_id = 0
    image_id = 0
    annotation_id = 0

    for video_name in os.listdir(base_path):
        video_path = osp.join(base_path, video_name)
        img_folder = osp.join(video_path, 'img')
        txt_file_path = osp.join(video_path, 'gt/gt.txt')

        if os.path.isdir(img_folder) and osp.exists(txt_file_path):
            video_id += 1
            video_info = {
                "id": video_id,
                "name": video_name,
                "fps": fps,
                "width": width,  # Gerekirse ayarlayın
                "height": height  # Gerekirse ayarlayın
            }

            is_test_set = video_name.endswith('4') and split_data
            if is_test_set:
                test_dataset["videos"].append(video_info)
            else:
                train_dataset["videos"].append(video_info)

            with open(txt_file_path, 'r') as f:
                frame_id = -1
                for line in f:
                    frame_id_txt, identity_id, x, y, w, h, score, object_category, time = line.strip().split(',')
                    frame_id_txt, identity_id, x, y, w, h, score, time = map(float, [frame_id_txt, identity_id, x, y, w, h, score, time])
                    object_category = int(object_category)

                    if frame_id_txt != frame_id:
                        frame_id = frame_id_txt
                        image_id += 1
                        image_name = f"{int(frame_id):06d}.jpg"
                        img_file_path = osp.join(img_folder, image_name)

                        if os.path.exists(img_file_path):
                            image_info = {
                                "id": image_id,
                                "video_id": video_id,
                                "file_name": osp.join(video_name, 'img', image_name).replace("\\", "/"),
                                "height": height,
                                "width": width,
                                "frame_id": int(frame_id-1),
                                "mot_frame_id": int(frame_id)
                            }

                            if is_test_set:
                                test_dataset["images"].append(image_info)
                            else:
                                train_dataset["images"].append(image_info)

                    annotation_id += 1
                    annotation_info = {
                        "id": annotation_id,
                        "image_id": image_id,
                        "category_id": 1,
                        "instance_id": int(identity_id),
                        "bbox": [x, y, w, h],
                        "area": w * h,
                        "iscrowd": False,
                        "visibility": 1.0,
                        "mot_instance_id": int(identity_id),
                        "mot_conf": score,
                        "mot_class_id": 1
                    }


                    if is_test_set:
                        test_dataset["annotations"].append(annotation_info)
                    else:
                        train_dataset["annotations"].append(annotation_info)

    train_dataset["num_instances"] = len(train_dataset["images"])
    test_dataset["num_instances"] = len(test_dataset["images"])

    with open('train_dataset.json', 'w') as train_file:
        json.dump(train_dataset, train_file, indent=4)

    with open('test_dataset.json', 'w') as test_file:
        json.dump(test_dataset, test_file, indent=4)

    return

In [27]:
base_path = 'data'
generate_dataset_structure(base_path, split_data=True)

({'categories': [{'id': 1, 'name': 'coconat'}],
  'annotations': [{'id': 1,
    'image_id': 1,
    'category_id': 1,
    'instance_id': 1,
    'bbox': [446.797, 249.602, 141.606, 141.602],
    'area': 20051.692812,
    'iscrowd': False,
    'visibility': 1.0,
    'mot_instance_id': 1,
    'mot_conf': 1.0,
    'mot_class_id': 1},
   {'id': 2,
    'image_id': 2,
    'category_id': 1,
    'instance_id': 1,
    'bbox': [446.797, 238.802, 141.606, 141.602],
    'area': 20051.692812,
    'iscrowd': False,
    'visibility': 1.0,
    'mot_instance_id': 1,
    'mot_conf': 1.0,
    'mot_class_id': 1},
   {'id': 3,
    'image_id': 3,
    'category_id': 1,
    'instance_id': 1,
    'bbox': [446.797, 227.398, 141.606, 141.602],
    'area': 20051.692812,
    'iscrowd': False,
    'visibility': 1.0,
    'mot_instance_id': 1,
    'mot_conf': 1.0,
    'mot_class_id': 1},
   {'id': 4,
    'image_id': 4,
    'category_id': 1,
    'instance_id': 1,
    'bbox': [446.797, 223.2, 141.606, 141.602],
    'area

In [28]:
import mmcv
from mmdet.apis import set_random_seed
cfg = mmcv.Config.fromfile('models/faster-rcnn_r50_fpn_4e_mot17-half.py')

cfg.work_dir = 'E:/internship_project/mmtracking-master/mmtracking-master/project-experiment/tez_projesi/result/det'
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)
print(f'Config:\n{cfg.pretty_text}')

Config:
model = dict(
    detector=dict(
        type='FasterRCNN',
        backbone=dict(
            type='ResNet',
            depth=50,
            num_stages=4,
            out_indices=(0, 1, 2, 3),
            frozen_stages=1,
            norm_cfg=dict(type='BN', requires_grad=True),
            norm_eval=True,
            style='pytorch',
            init_cfg=dict(
                type='Pretrained', checkpoint='torchvision://resnet50')),
        neck=dict(
            type='FPN',
            in_channels=[256, 512, 1024, 2048],
            out_channels=256,
            num_outs=5),
        rpn_head=dict(
            type='RPNHead',
            in_channels=256,
            feat_channels=256,
            anchor_generator=dict(
                type='AnchorGenerator',
                scales=[8],
                ratios=[0.5, 1.0, 2.0],
                strides=[4, 8, 16, 32, 64]),
            bbox_coder=dict(
                type='DeltaXYWHBBoxCoder',
                target_means=[0.0,

In [29]:
import mmcv
from mmdet.apis import set_random_seed
from mmtrack.datasets import build_dataset
from mmdet.apis import train_detector as train_model
from mmdet.models import build_detector as build_model
import torch
cfg = mmcv.Config.fromfile('models/faster-rcnn_r50_fpn_4e_mot17-half.py')

cfg.work_dir = 'E:/internship_project/mmtracking-master/mmtracking-master/project-experiment/tez_projesi/result/det'
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)
print(f'Config:\n{cfg.pretty_text}')

torch.cuda.empty_cache()
mmcv.mkdir_or_exist(cfg.work_dir)
model = build_model(cfg.model.detector)
model.init_weights()
datasets = [build_dataset(cfg.data.train)]
model.CLASSES = datasets[0].CLASSES
train_model(model, datasets, cfg)

2024-06-07 12:22:59,564 - mmcv - INFO - initialize FasterRCNN with init_cfg {'type': 'Pretrained', 'checkpoint': 'http://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_2x_coco/faster_rcnn_r50_fpn_2x_coco_bbox_mAP-0.384_20200504_210434-a5d8aa15.pth'}
2024-06-07 12:22:59,565 - mmcv - INFO - load model from: http://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_2x_coco/faster_rcnn_r50_fpn_2x_coco_bbox_mAP-0.384_20200504_210434-a5d8aa15.pth
2024-06-07 12:22:59,566 - mmcv - INFO - load checkpoint from http path: http://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_2x_coco/faster_rcnn_r50_fpn_2x_coco_bbox_mAP-0.384_20200504_210434-a5d8aa15.pth
2024-06-07 12:22:59,701 - mmcv - WARNING - The model and loaded state dict do not match exactly

size mismatch for roi_head.bbox_head.fc_cls.weight: copying a param with shape torch.Size([81, 1024]) from checkpoint, the shape in current model is torch.Size([2, 1024]).
size mism

loading annotations into memory...
Done (t=0.10s)
creating index...
index created!


2024-06-07 12:23:00,005 - mmdet - INFO - Automatic scaling of learning rate (LR) has been disabled.
2024-06-07 12:23:00,008 - mmdet - INFO - Start running, host: bmb_o@DESKTOP-ONS0OK4, work_dir: E:\internship_project\mmtracking-master\mmtracking-master\project-experiment\tez_projesi\result\det
2024-06-07 12:23:00,009 - mmdet - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) CheckpointHook                     
(VERY_LOW    ) TextLoggerHook                     
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) StepLrUpdaterHook                  
(LOW         ) IterTimerHook                      
(VERY_LOW    ) TextLoggerHook                     
 -------------------- 
before_train_iter:
(VERY_HIGH   ) StepLrUpdaterHook                  
(LOW         ) IterTimerHook                      
 -------------------- 
after_train_iter:
(ABOVE_NORMAL) OptimizerHook                      
(NORMAL      ) Check

KeyboardInterrupt: 

In [46]:
!python tools/test.py project-experiment/tez_projesi/models/faster-rcnn_r50_fpn_4e_mot17-half.py --checkpoint project-experiment/tez_projesi/result/det/epoch_4.pth --eval bbox

python: can't open file 'tools/test.py': [Errno 2] No such file or directory


In [5]:
import os
import os.path as osp
import mmcv
import numpy as np
import random
from collections import defaultdict
from tqdm import tqdm

def generate_reid_dataset(base_path, output_path, val_split=0.2, min_object=1, max_object=50, vis_threshold=0.0):
    if not osp.isdir(output_path):
        os.makedirs(output_path)
    elif os.listdir(output_path):
        raise OSError(f'Directory must be empty: \'{output_path}\'')

    video_names = os.listdir(base_path)
    reid_train_folder = osp.join(output_path, 'imgs')
    if not osp.exists(reid_train_folder):
        os.makedirs(reid_train_folder)

    for video_name in tqdm(video_names):
        video_folder = osp.join(base_path, video_name)
        img_folder = osp.join(video_folder, 'img')
        data_file_path = osp.join(video_folder, 'gt/gt.txt')
        if not osp.exists(data_file_path):
            continue

        raw_img_names = sorted(os.listdir(img_folder))
        data_lines = mmcv.list_from_file(data_file_path)

        last_frame_id = -1
        for line in data_lines:
            line = line.strip().split(',')
            frame_id, ins_id = map(int, line[:2])
            x, y, w, h = map(float, line[2:6])
            confidence = float(line[6])
            # object_category ve time kullanılmıyor, bu yüzden atlanabilir

            if confidence < vis_threshold:
                continue

            reid_img_folder = osp.join(reid_train_folder, f'{video_name}_{ins_id:06d}')
            if not osp.exists(reid_img_folder):
                os.makedirs(reid_img_folder)
            idx = len(os.listdir(reid_img_folder))
            reid_img_name = f'{idx:06d}.jpg'
            if frame_id != last_frame_id:
                raw_img_name = raw_img_names[frame_id - 1]
                raw_img = mmcv.imread(f'{img_folder}/{raw_img_name}')
                last_frame_id = frame_id
            xyxy = np.asarray([x, y, x + w, y + h])
            reid_img = mmcv.imcrop(raw_img, xyxy)
            mmcv.imwrite(reid_img, f'{reid_img_folder}/{reid_img_name}')

    reid_meta_folder = osp.join(output_path, 'meta')
    if not osp.exists(reid_meta_folder):
        os.makedirs(reid_meta_folder)
    reid_train_list = []
    reid_val_list = []
    reid_img_folder_names = sorted(os.listdir(reid_train_folder))
    num_ids = len(reid_img_folder_names)
    num_train_ids = int(num_ids * (1 - val_split))
    train_label, val_label = 0, 0
    random.seed(0)

    for reid_img_folder_name in reid_img_folder_names[:num_train_ids]:
        reid_img_names = os.listdir(f'{reid_train_folder}/{reid_img_folder_name}')
        if len(reid_img_names) < min_object:
            continue
        if len(reid_img_names) > max_object:
            reid_img_names = random.sample(reid_img_names, max_object)
        for reid_img_name in reid_img_names:
            reid_train_list.append(f'{reid_img_folder_name}/{reid_img_name} {train_label}\n')
        train_label += 1

    reid_entire_dataset_list = reid_train_list.copy()
    for reid_img_folder_name in reid_img_folder_names[num_train_ids:]:
        reid_img_names = os.listdir(f'{reid_train_folder}/{reid_img_folder_name}')
        if len(reid_img_names) < min_object:
            continue
        if len(reid_img_names) > max_object:
            reid_img_names = random.sample(reid_img_names, max_object)
        for reid_img_name in reid_img_names:
            reid_val_list.append(f'{reid_img_folder_name}/{reid_img_name} {val_label}\n')
            reid_entire_dataset_list.append(f'{reid_img_folder_name}/{reid_img_name} {train_label + val_label}\n')
        val_label += 1

    with open(osp.join(reid_meta_folder, f'train_{int(100 * (1 - val_split))}.txt'), 'w') as f:
        f.writelines(reid_train_list)
    with open(osp.join(reid_meta_folder, f'val_{int(100 * val_split)}.txt'), 'w') as f:
        f.writelines(reid_val_list)
    with open(osp.join(reid_meta_folder, 'train.txt'), 'w') as f:
        f.writelines(reid_entire_dataset_list)

E:\internship_project\mmtracking-master\mmtracking-master\env\lib\site-packages\mmcv\__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(


In [31]:
base_path = 'data'
output_path = 'reid'
generate_reid_dataset(base_path, output_path, val_split=0.2)

  0%|          | 0/6 [00:00<?, ?it/s]

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


 17%|█▋        | 1/6 [00:05<00:25,  5.02s/it]

286
287
288
289
290
291
292
293
294
295
296
297
298
299
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156


 33%|███▎      | 2/6 [00:07<00:14,  3.52s/it]

...

 67%|██████▋   | 4/6 [00:18<00:09,  4.97s/it]

299
300
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275


 83%|████████▎ | 5/6 [00:26<00:05,  5.85s/it]

295
296
297
298
299
300
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271


100%|██████████| 6/6 [00:33<00:00,  5.64s/it]


In [32]:
import mmcv
from mmdet.apis import set_random_seed
cfg = mmcv.Config.fromfile('models/resnet50_b32x8_MOT17.py')
cfg.lr_config = dict(
    policy='step',
    warmup='linear',
    warmup_iters=200,
    warmup_ratio=1.0 / 200,
    step=[1])

cfg.work_dir = 'E:/internship_project/mmtracking-master/mmtracking-master/project-experiment/tez_projesi/result/reid'
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)
cfg.device ='cuda'
print(f'Config:\n{cfg.pretty_text}')

Config:
dataset_type = 'ReIDDataset'
img_norm_cfg = dict(
    mean=[123.675, 116.28, 103.53], std=[58.395, 57.12, 57.375], to_rgb=True)
train_pipeline = [
    dict(type='LoadMultiImagesFromFile', to_float32=True),
    dict(
        type='SeqResize',
        img_scale=(128, 256),
        share_params=False,
        keep_ratio=False,
        bbox_clip_border=False,
        override=False),
    dict(
        type='SeqRandomFlip',
        share_params=False,
        flip_ratio=0.5,
        direction='horizontal'),
    dict(
        type='SeqNormalize',
        mean=[123.675, 116.28, 103.53],
        std=[58.395, 57.12, 57.375],
        to_rgb=True),
    dict(type='VideoCollect', keys=['img', 'gt_label']),
    dict(type='ReIDFormatBundle')
]
test_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='Resize', img_scale=(128, 256), keep_ratio=False),
    dict(
        type='Normalize',
        mean=[123.675, 116.28, 103.53],
        std=[58.395, 57.12, 57.375],
        to_rgb=True),

In [33]:
from mmtrack.datasets import build_dataset
from mmdet.apis import train_detector as train_model
from mmtrack.models import build_reid as build_model


model = build_model(cfg.model.reid)
model.init_weights()
datasets = [build_dataset(cfg.data.train)]
model.CLASSES = datasets[0].CLASSES
train_model(model, datasets, cfg)

2024-06-07 12:27:01,988 - mmcv - INFO - initialize BaseReID with init_cfg {'type': 'Pretrained', 'checkpoint': 'https://download.openmmlab.com/mmclassification/v0/resnet/resnet50_batch256_imagenet_20200708-cfb998bf.pth'}
2024-06-07 12:27:01,990 - mmcv - INFO - load model from: https://download.openmmlab.com/mmclassification/v0/resnet/resnet50_batch256_imagenet_20200708-cfb998bf.pth
2024-06-07 12:27:01,990 - mmcv - INFO - load checkpoint from http path: https://download.openmmlab.com/mmclassification/v0/resnet/resnet50_batch256_imagenet_20200708-cfb998bf.pth
2024-06-07 12:27:02,119 - mmcv - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: head.fc.weight, head.fc.bias

missing keys in source state_dict: head.fcs.0.fc.weight, head.fcs.0.fc.bias, head.fcs.0.bn.weight, head.fcs.0.bn.bias, head.fcs.0.bn.running_mean, head.fcs.0.bn.running_var, head.fc_out.weight, head.fc_out.bias, head.bn.weight, head.bn.bias, head.bn.running_mean, head.bn.

KeyboardInterrupt: 

In [57]:
!python tools/test.py project-experiment/tez_projesi/models/resnet50_b32x8_MOT17.py --checkpoint project-experiment/tez_projesi/result/reid/epoch_2.pth --eval mAP

In [6]:
import mmcv
import tempfile
from mmtrack.apis import inference_mot, init_model

E:\internship_project\mmtracking-master\mmtracking-master\env\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [11]:
def process_frames(input_dir, mot_config, output_video ,device='cuda:0', fps=30):
    frames = sorted([f for f in os.listdir(input_dir) if f.endswith('.jpg') or f.endswith('.png')])
    num_frames = len(frames)

    mot_model = init_model(mot_config,device=device)
    prog_bar = mmcv.ProgressBar(num_frames)

    out_dir = tempfile.TemporaryDirectory()
    out_path = out_dir.name

    for i, frame_file in enumerate(frames):
        img_path = os.path.join(input_dir, frame_file)
        img = cv2.imread(img_path)

        result = inference_mot(mot_model, img, frame_id=i)
        print(result)

        mot_model.show_result(
                img,
                result,
                show=True,
                wait_time=int(1000. / fps),
                out_file=f'{out_path}/{i:06d}.jpg')
        prog_bar.update()


    print(f'\n making the output video at {output_video} with a FPS of {fps}')
    mmcv.frames2video(out_path, output_video, fps=fps, fourcc='mp4v')
    out_dir.cleanup()


In [1]:
import cv2
import mmcv
import os
import tempfile
import numpy as np
from mmtrack.apis import inference_mot, init_model

def process_frames(input_dir, mot_config, output_video, device='cuda:0', fps=30):
    frames = sorted([f for f in os.listdir(input_dir) if f.endswith('.jpg') or f.endswith('.png')])
    num_frames = len(frames)

    mot_model = init_model(mot_config, device=device)
    prog_bar = mmcv.ProgressBar(num_frames)

    out_dir = tempfile.TemporaryDirectory()
    out_path = out_dir.name

    count = 0
    areas = []
    tracked_ids=set()

    for i, frame_file in enumerate(frames):
        img_path = os.path.join(input_dir, frame_file)
        img = cv2.imread(img_path)
        height, width, _ = img.shape
        mid_line_y = height // 2

        result = inference_mot(mot_model, img, frame_id=i)
        mot_model.show_result(
            img,
            result,
            show=False,
            wait_time=int(1000. / fps),
            out_file=f'{out_path}/{i:06d}.jpg')

        # Check for objects crossing the middle line
        for bbox in result['track_bboxes'][0]:
            if isinstance(bbox, np.ndarray) and bbox.ndim == 1 and len(bbox) >= 4:
                track_id,xmin, ymin, bbox_width, bbox_height = bbox[:5]
                xmax = xmin + bbox_width
                ymax = ymin + bbox_height
                if ymin <= mid_line_y <= ymax:
                    if track_id not in tracked_ids:
                      tracked_ids.add(track_id)
                      count += 1
                      bbox_area = bbox_width * bbox_height
                      areas.append(bbox_area)

        prog_bar.update()



    print(f'\nMaking the output video at {output_video} with a FPS of {fps}')
    mmcv.frames2video(out_path, output_video, fps=fps, fourcc='mp4v')
    print(f'\nTotal objects crossed the line: {count}')
    out_dir.cleanup()
    return
cfg = mmcv.Config.fromfile('models/deepsort_faster-rcnn_fpn_4e_mot17-private-half.py')
cfg.model.detector.init_cfg.checkpoint = 'E:/internship_project/mmtracking-master/mmtracking-master/project-experiment/tez_projesi/result/det/epoch_4.pth'
cfg.model.reid.init_cfg.checkpoint = 'E:/internship_project/mmtracking-master/mmtracking-master/project-experiment/tez_projesi/result/reid/epoch_6.pth'
input_dir = 'E:/internship_project/mmtracking-master/mmtracking-master/project-experiment/tez_projesi/data/segment_4/img' # Frame'lerin bulunduğu dizin
output_video = 'output_video.mp4'  # Oluşturulacak video dosyasının adı
process_frames(input_dir=input_dir,mot_config=cfg,output_video=output_video)

E:\internship_project\mmtracking-master\mmtracking-master\env\lib\site-packages\mmcv\__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(
E:\internship_project\mmtracking-master\mmtracking-master\env\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
2024-06-25 08:51:49,532 - mmcv - INFO - initialize FasterRCNN with init_cfg {'type': 'Pretrained', 'checkpoint': 'E:/internship_project/mmtracking-master/mmtracking-master/project-experiment/tez_projesi/result/det/epoch_4.pth'}
2024-06-25 08:51:49

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 300/300, 3.0 task/s, elapsed: 100s, ETA:     0s
Total objects crossed the line: 32

Making the output video at output_video.mp4 with a FPS of 30
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 300/300, 42.1 task/s, elapsed: 7s, ETA:     0s


In [74]:
import cv2
import mmcv
import os
from sklearn.cluster import KMeans
import numpy as npnp

def detect_size(input_dir, mot_config, device='cuda:0', fps=30):
    # List and sort the image files in the given input directory
    frames = sorted([f for f in os.listdir(input_dir) if f.endswith('.jpg') or f.endswith('.png')])
    num_frames = len(frames)

    # Initialize the tracking model with the specified configuration and device
    mot_model = init_model(mot_config, device=device)
    prog_bar = mmcv.ProgressBar(num_frames)
    count = 0
    areas = []
    tracked_ids = set()

    # Process each frame in the directory
    for i, frame_file in enumerate(frames):
        img_path = os.path.join(input_dir, frame_file)
        img = cv2.imread(img_path)
        height, width, _ = img.shape
        mid_line_y = height // 2  # Calculate the middle line of the image

        # Perform inference using the tracking model
        result = inference_mot(mot_model, img, frame_id=i)

        for bbox in result['track_bboxes'][0]:  # Assuming result['track_bboxes'] is a list of arrays
            if isinstance(bbox, np.ndarray) and bbox.ndim == 1 and len(bbox) >= 4:
                track_id, xmin, ymin, bbox_width, bbox_height = bbox[:5]
                xmax = xmin + bbox_width
                ymax = ymin + bbox_height
                # Check if the bbox crosses the middle horizontal line
                if ymin <= mid_line_y <= ymax:
                    if track_id not in tracked_ids:
                        tracked_ids.add(track_id)  # Track only unique IDs crossing the line
                        count += 1
                        bbox_area = bbox_width * bbox_height  # Calculate the area of the bbox
                        areas.append(bbox_area)  # Store the area

        prog_bar.update()  # Update the progress bar for visual feedback
    X = np.array(areas).reshape(-1, 1)
    kmeans = KMeans(n_clusters=2, random_state=0).fit(X)
    centers = kmeans.cluster_centers_.squeeze()
    order = centers.argsort()
    labels = order[kmeans.labels_]

    group_large = [areas[i] for i in range(len(areas)) if labels[i] == order[0]]
    group_small = [areas[i] for i in range(len(areas)) if labels[i] == order[1]]

    range_small = (int(min(group_small)), int(max(group_small)))
    range_large = (int(max(group_small)), int(max(group_large)))

    count_small = len(group_small)
    count_large = len(group_large)

    # Output results
    print(f'Total objects crossed the line: {len(tracked_ids)}')
    print(f'Number of small objects: {count_small}')
    print(f'Number of large objects: {count_large}')
    print(f'Small group range: {range_small}')
    print(f'Large group range: {range_large}')

    # Clean up temporary directory and create video

In [58]:
import cv2
import mmcv
import os
import numpy as np
import json
from tempfile import TemporaryDirectory

def draw_label_on_image(image, bbox, label):
    # Draw label on the image at bbox location
    x, y, w, h = bbox
    cv2.putText(image, label, (int(x), int(y) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
    return image

def detect_size(input_dir, mot_config, output_video, json_path, device='cuda:0', fps=30):
    # Load size ranges from JSON
    with open(json_path, 'r') as f:
        size_ranges = json.load(f)

    frames = sorted([f for f in os.listdir(input_dir) if f.endswith('.jpg') or f.endswith('.png')])
    num_frames = len(frames)

    mot_model = init_model(mot_config, device=device)
    prog_bar = mmcv.ProgressBar(num_frames)
    out_dir = tempfile.TemporaryDirectory()
    out_path = out_dir.name
    label_counts = {label: 0 for label in size_ranges}
    tracked_ids = {}

    # Create a temporary directory to store processed images
    with TemporaryDirectory() as temp_dir:
        for i, frame_file in enumerate(frames):
            img_path = os.path.join(input_dir, frame_file)
            img = cv2.imread(img_path)
            height, width, _ = img.shape
            mid_line_y = height // 2  # Calculate the middle line of the image
            out_path_img = f'{out_path}/{i:06d}.jpg'
            result = inference_mot(mot_model, img, frame_id=i)
            mot_model.show_result(
            img,
            result,
            show=False,
            wait_time=int(1000. / fps),
            out_file=out_path_img)
            img_bb = cv2.imread(out_path_img)

            for bbox in result['track_bboxes'][0]:
                if isinstance(bbox, np.ndarray) and bbox.ndim == 1 and len(bbox) >= 5:
                    track_id, xmin, ymin, bbox_width, bbox_height = bbox[:5]
                    diagonal = np.sqrt(bbox_width ** 2 + bbox_height ** 2)

                    label = "Undefined"
                    for size_label, (min_diag, max_diag) in size_ranges.items():
                        if min_diag <= diagonal <= max_diag:
                            label = size_label
                            break

                    xmax = xmin + bbox_width
                    ymax = ymin + bbox_height
                    if ymin <= mid_line_y <= ymax:
                        if track_id not in tracked_ids.keys():
                            tracked_ids[track_id] = label  # Update or add track_id with its label
                            label_counts[label] += 1
                        img_bb = draw_label_on_image(img_bb, (xmin, ymin, bbox_width, bbox_height), label)
                        print(label_counts)

            cv2.imwrite(out_path_img, img_bb)
            prog_bar.update()

        # Use mmcv to convert images to video
        mmcv.frames2video(out_path, output_video, fps=fps, fourcc='mp4v')

    print("Processing complete. Video saved to", output_video)

In [66]:
import cv2
import os

def frames_to_video(input_folder, output_video_path, fps=30):
    # Get all folder names in the input directory
    folders = sorted([f for f in os.listdir(input_folder) if os.path.isdir(os.path.join(input_folder, f))])

    for i in range(1):
        frame_files = sorted([os.path.join(input_folder, f) for f in os.listdir(os.path.join(input_folder)) if f.endswith('.jpg')])

        if len(frame_files) == 0:
            continue

        # Read the first frame to get the frame size
        frame = cv2.imread(frame_files[0])
        height, width, layers = frame.shape

        # Initialize the video writer
        output_video = cv2.VideoWriter(f"{output_video_path}", cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

        for frame_file in frame_files:
            frame = cv2.imread(frame_file)
            output_video.write(frame)

        output_video.release()

In [67]:
frames_to_video('data/segment_4/img','test.mp4')

In [59]:
cfg = mmcv.Config.fromfile('models/deepsort_faster-rcnn_fpn_4e_mot17-private-half.py')
cfg.model.detector.init_cfg.checkpoint = 'E:/internship_project/mmtracking-master/mmtracking-master/project-experiment/tez_projesi/result/det/epoch_4.pth'
cfg.model.reid.init_cfg.checkpoint = 'E:/internship_project/mmtracking-master/mmtracking-master/project-experiment/tez_projesi/result/reid/epoch_6.pth'
input_dir = 'E:/internship_project/mmtracking-master/mmtracking-master/project-experiment/tez_projesi/data/segment_4/img' # Frame'lerin bulunduğu dizin
mot_config = 'models/deepsort_faster-rcnn_fpn_4e_mot17-private-half.py'  # Model konfigürasyon dosyası
output_video = 'output2_video.mp4'  # Oluşturulacak video dosyasının adı

In [60]:
detect_size(input_dir=input_dir, mot_config=cfg, output_video=output_video, json_path="size.json", fps=30)

2024-06-25 14:13:32,399 - mmcv - INFO - initialize FasterRCNN with init_cfg {'type': 'Pretrained', 'checkpoint': 'E:/internship_project/mmtracking-master/mmtracking-master/project-experiment/tez_projesi/result/det/epoch_4.pth'}
2024-06-25 14:13:32,401 - mmcv - INFO - load model from: E:/internship_project/mmtracking-master/mmtracking-master/project-experiment/tez_projesi/result/det/epoch_4.pth
2024-06-25 14:13:32,402 - mmcv - INFO - load checkpoint from local path: E:/internship_project/mmtracking-master/mmtracking-master/project-experiment/tez_projesi/result/det/epoch_4.pth
2024-06-25 14:13:33,594 - mmcv - INFO - initialize BaseReID with init_cfg {'type': 'Pretrained', 'checkpoint': 'E:/internship_project/mmtracking-master/mmtracking-master/project-experiment/tez_projesi/result/reid/epoch_6.pth'}
2024-06-25 14:13:33,595 - mmcv - INFO - load model from: E:/internship_project/mmtracking-master/mmtracking-master/project-experiment/tez_projesi/result/reid/epoch_6.pth
2024-06-25 14:13:33,5

[                                                  ] 0/300, elapsed: 0s, ETA:{'Small': 0, 'Medium': 0, 'Large': 1, 'Extra Large': 0, 'Undefined': 0}
[                                 ] 1/300, 5.2 task/s, elapsed: 0s, ETA:    57s{'Small': 0, 'Medium': 0, 'Large': 1, 'Extra Large': 0, 'Undefined': 0}
{'Small': 0, 'Medium': 0, 'Large': 2, 'Extra Large': 0, 'Undefined': 0}
[                                 ] 2/300, 5.3 task/s, elapsed: 0s, ETA:    56s{'Small': 0, 'Medium': 0, 'Large': 2, 'Extra Large': 0, 'Undefined': 0}
{'Small': 0, 'Medium': 0, 'Large': 2, 'Extra Large': 0, 'Undefined': 0}
[                                 ] 3/300, 5.2 task/s, elapsed: 1s, ETA:    57s{'Small': 0, 'Medium': 0, 'Large': 2, 'Extra Large': 0, 'Undefined': 0}
{'Small': 0, 'Medium': 0, 'Large': 2, 'Extra Large': 0, 'Undefined': 0}
[                                 ] 4/300, 5.2 task/s, elapsed: 1s, ETA:    56s{'Small': 0, 'Medium': 0, 'Large': 2, 'Extra Large': 0, 'Undefined': 0}
{'Small': 0, 'Medium': 0, 'Larg

In [44]:
detect_size(input_dir=input_dir,mot_config= cfg)

In [45]:
import cv2
import mmcv

2024-06-11 15:50:26,328 - mmcv - INFO - initialize FasterRCNN with init_cfg {'type': 'Pretrained', 'checkpoint': 'E:/internship_project/mmtracking-master/mmtracking-master/project-experiment/tez_projesi/result/det/epoch_4.pth'}
2024-06-11 15:50:26,329 - mmcv - INFO - load model from: E:/internship_project/mmtracking-master/mmtracking-master/project-experiment/tez_projesi/result/det/epoch_4.pth
2024-06-11 15:50:26,330 - mmcv - INFO - load checkpoint from local path: E:/internship_project/mmtracking-master/mmtracking-master/project-experiment/tez_projesi/result/det/epoch_4.pth
2024-06-11 15:50:26,976 - mmcv - INFO - initialize BaseReID with init_cfg {'type': 'Pretrained', 'checkpoint': 'E:/internship_project/mmtracking-master/mmtracking-master/project-experiment/tez_projesi/result/reid/epoch_6.pth'}
2024-06-11 15:50:26,977 - mmcv - INFO - load model from: E:/internship_project/mmtracking-master/mmtracking-master/project-experiment/tez_projesi/result/reid/epoch_6.pth
2024-06-11 15:50:26,9

[                                                  ] 0/300, elapsed: 0s, ETA:[  0.         569.31951904  35.71598053 701.01397705 180.94290161
   0.99977142]
[  1.         350.23501587 249.2644043  498.57281494 388.05413818
   0.99967504]
[  2.         570.82092285 365.91543579 721.83209229 518.00842285
   0.99950707]
[  3.         474.03460693 544.10083008 621.3895874  702.54302979
   0.99912614]
[4.00000000e+00 4.71880707e+02 5.56494117e-01 6.20362732e+02
 9.54718094e+01 9.98159230e-01]
[  5.         711.89831543 418.92407227 847.95227051 557.75378418
   0.99358189]
[                                 ] 1/300, 2.0 task/s, elapsed: 1s, ETA:   151s[  1.         350.72897339 241.51077271 500.44403076 376.08972168
   0.99974281]
[  0.         568.11431885  30.70924759 700.15942383 170.36430359
   0.99974185]
[  2.         570.40203857 359.48861694 721.54016113 511.24282837
   0.99948692]
[  3.         474.27822876 534.31616211 619.67608643 692.29040527
   0.99908841]
[4.00000000e+00 4.7084

In [14]:
import mmcv
from mmdet.apis import set_random_seed
cfg = mmcv.Config.fromfile('models/deepsort_faster-rcnn_fpn_4e_mot17-private-half.py')
cfg.model.detector.init_cfg.checkpoint = 'E:/internship_project/mmtracking-master/mmtracking-master/project-experiment/tez_projesi/result/det/epoch_1.pth'
cfg.model.reid.init_cfg.checkpoint = 'E:/internship_project/mmtracking-master/mmtracking-master/project-experiment/tez_projesi/result/reid/epoch_2.pth'
cfg.work_dir = 'E:/internship_project/mmtracking-master/mmtracking-master/project-experiment/tez_projesi/result/deepsort'
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)
cfg.data.test.test_mode = True
print(f'Config:\n{cfg.pretty_text}')

In [19]:
from mmtrack.datasets import build_dataloader
from mmtrack.apis import init_model
from mmcv.parallel import MMDataParallel
from mmtrack.apis import single_gpu_test
from mmtrack.datasets import build_dataset

dataset = build_dataset(cfg.data.test)
data_loader = build_dataloader(
    dataset,
    samples_per_gpu=1,
    workers_per_gpu=cfg.data.workers_per_gpu,
    dist=False,
    shuffle=False)

# build the model and load checkpoint
model = init_model(cfg)

model = MMDataParallel(model, device_ids=cfg.gpu_ids)
outputs = single_gpu_test(model, data_loader)

eval_kwargs = cfg.get('evaluation', {}).copy()
# hard-code way to remove EvalHook args
eval_hook_args = [
    'interval', 'tmpdir', 'start', 'gpu_collect', 'save_best',
    'rule', 'by_epoch'
]
for key in eval_hook_args:
    eval_kwargs.pop(key, None)
eval_kwargs.update(dict(metric=['track']))
metric = dataset.evaluate(outputs, **eval_kwargs)
print(metric)

loading annotations into memory...
Done (t=0.02s)
creating index...
index created!


2024-06-07 12:15:19,931 - mmcv - INFO - initialize FasterRCNN with init_cfg {'type': 'Pretrained', 'checkpoint': 'E:/internship_project/mmtracking-master/mmtracking-master/project-experiment/tez_projesi/result/det/epoch_1.pth'}
INFO:mmcv:initialize FasterRCNN with init_cfg {'type': 'Pretrained', 'checkpoint': 'E:/internship_project/mmtracking-master/mmtracking-master/project-experiment/tez_projesi/result/det/epoch_1.pth'}
2024-06-07 12:15:19,933 - mmcv - INFO - load model from: E:/internship_project/mmtracking-master/mmtracking-master/project-experiment/tez_projesi/result/det/epoch_1.pth
INFO:mmcv:load model from: E:/internship_project/mmtracking-master/mmtracking-master/project-experiment/tez_projesi/result/det/epoch_1.pth
2024-06-07 12:15:19,934 - mmcv - INFO - load checkpoint from local path: E:/internship_project/mmtracking-master/mmtracking-master/project-experiment/tez_projesi/result/det/epoch_1.pth
INFO:mmcv:load checkpoint from local path: E:/internship_project/mmtracking-maste

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 300/300, 6.1 task/s, elapsed: 49s, ETA:     0sEvaluate CLEAR MOT results.

Eval Config:
USE_PARALLEL         : False                         
NUM_PARALLEL_CORES   : 8                             
BREAK_ON_ERROR       : True                          
RETURN_ON_ERROR      : False                         
LOG_ON_ERROR         : E:\internship_project\mmtracking-master\mmtracking-master\env\lib\site-packages\error_log.txt
PRINT_RESULTS        : True                          
PRINT_ONLY_COMBINED  : False                         
PRINT_CONFIG         : True                          
TIME_PROGRESS        : True                          
DISPLAY_LESS_PROGRESS : True                          
OUTPUT_SUMMARY       : True                          
OUTPUT_EMPTY_CLASSES : True                          
OUTPUT_DETAILED      : True                          
PLOT_CURVES          : True                          

MotChallenge2DBox Config:
GT_FOLDER            : C:\Users\

<Figure size 640x480 with 0 Axes>

E:\internship_project\mmtracking-master\mmtracking-master\env\lib\site-packages\mmcv\__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(
